# Logistic Regression

For this problem, you will be implementing a Logistic Regression model for binary classification (yes-or-no like predictions). This is a simple model with restricted application areas, but it is very useful when applied to appropriate problems. Moreover, for those of you interested in Deep Learning, many of the concepts covered here are fundamental to how most neural networks as built and optimized.

You should begin by implementing a standard form of logistic regression. That is, we want to predict a target variable $y \in \{0, 1\}$ given observed feature variables $x \in \mathbb{R}^n$, and their relationship is modeled as:

- prediction $ = \hat{y} = h_\theta(x) = \sigma(\theta^T x) \in (0, 1)$, where:
    - $h_\theta$ is your Logistic Regression model
    - $\theta \in \mathbb{R}^n$ is the parameters of the model
    - $\sigma(z) = \frac{1}{1 + e^{-z}}$ is the logistic function (`sigmoid` in the provided code)

The parameters $\theta$ should be optimized with gradient descent on a loss derived from maximum likelihood estimation under the assumption that $P(y | x, \theta)$ is Bernoulli distributed.
Please refer to [Andrew Ng's lecture notes on supervised learning](http://cs229.stanford.edu/notes2020spring/cs229-notes1.pdf) for more details. The most salient information for this problem can be found in Section 5, but we recommend that you read through all of it.

We have provided some skeleton code for the classifier, along with a couple of utility functions in the [logistic_regression.py](./logistic_regression.py) module. Please fill out the functions marked with `TODO` and feel free to add extra constructor arguments as you see fit (just make sure the default constructor solves the first dataset).

In [ ]:
%load_ext autoreload

We begin by loading necessary packages. Below follows a short description of the imported modules:

- `numpy` is the defacto python package for numerical calculation. Most other numerical libraries (including pandas) is based on numpy.
- `pandas` is a widely used package for manipulating (mostly) tabular data
- `matplotlib` is the most used plotting library for python
- `seaborn` contains several convience functions for matplotlib and integrates very well with pandas
- `logistic_regression` refers to the module in this folder that should be further implemented by you.

Note: The `%autoreload` statement is an [IPython magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html) that automatically reloads the newest version of all imported modules within the cell. This means that you can edit the `logistic_regression.py` file and just rerun this cell to get the updated version.

In [ ]:
%autoreload 

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import logistic_regression as lr   # <-- Your implementation

sns.set_style('darkgrid') # Seaborn plotting style 

## [1] First Dataset

The first dataset is a simple problem that can be used to debug your algorithm. The objective is to predict whether the dependent variable ($y$) should be equal to 1 or 0 based on two independent variables ($x_0$ and $x_1$).

### [1.1] Load Data

We begin by loading data from the .csv file located in the same folder as this notebook.

In [ ]:
data_1 = pd.read_csv('data_1.csv')
data_1.head()

### [1.2] Visualize

Since the feature-space is continuous and 2-dimensional, it lends itself nicely to visualization with a scatter plot.

In [ ]:
plt.figure(figsize=(5, 5))
sns.scatterplot(x='x0', y='x1', hue='y', data=data_1);

### [1.3] Train and Evaluate

Next we fit and evaluate a Logistic Regression classifier over the dataset. We first partition the data into the dependent (`y`) and independent (`X`) variables. We then initialize a Logistic Regression classifier and fit it to all the data. Finally, we evaluate the model over the same data by calculating _accuracy_ and (binary) _cross entropy_.

- The accuracy is given by the fraction of correctly classified samples. Since the Logistic Regression classifier outputs "soft" predictions ($\hat{y} \in (0, 1)$), we threshold the predictions so that $\hat{y} \geq 0.5$ is considered a $1$ and $\hat{y} < 0.5$ is considered a $0$.

- The [cross entropy](https://en.wikipedia.org/wiki/Cross_entropy) measures the dissimilarity between the distributions of the true labels ($y$) and the "label-probablities" ($\hat{y}$) predicted by your model. It is subject to minimization and (hint) _highly_ related to your maximal likelihood objective.

Note that `.fit` and `.predict` will crash until you implement these two methods in [logistic_regression.py](./logistic_regression.py).

Assuming a standard implementation of logistic regression with batch gradient descent, you should expect to get an accuracy of at least 90%.

In [ ]:
# Partition data into independent (feature) and depended (target) variables
X = data_1[['x0', 'x1']]
y = data_1['y']

# Create and train model.
model_1 = lr.LogisticRegression() # <-- Should work with default constructor  
model_1.fit(X, y)

# Calculate accuracy and cross entropy for (insample) predictions 
y_pred = model_1.predict(X)
print(f'Accuracy: {lr.binary_accuracy(y_true=y, y_pred=y_pred, threshold=0.5) :.3f}')
print(f'Cross Entropy: {lr.binary_cross_entropy(y_true=y, y_pred=y_pred) :.3f}')

### [1.4] Visualize Decision Boundary

Logistic Regression does not make hard classification decisions. Instead, it attempts to model the probability of a datapoint belonging to the 1-class after conditioning on the available features ($x_0, x_1$ here). The modeled probability of the 0-class can be obtained by negating and adding one.

We can visualize the modeled probabilities across the entire input space. In the cell below, we rasterize the model's prediction over a grid that covers approximately the same area as the data used to train it. Red regions correspond to high modeled probability for the 1-class whereas blue correspond to high modeled probability for the 0-class. Does the decisions of your model line up well with the provided data? What happens if you train it for longer?

In [ ]:
# Rasterize the model's predictions over a grid
xx0, xx1 = np.meshgrid(np.linspace(-0.1, 1.1, 100), np.linspace(-0.1, 1.1, 100))
yy = model_1.predict(np.stack([xx0, xx1], axis=-1).reshape(-1, 2)).reshape(xx0.shape)

# Plot prediction countours along with datapoints
_, ax = plt.subplots(figsize=(4, 4), dpi=100)
levels = [0, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 1.0]
contours = ax.contourf(xx0, xx1, yy, levels=levels, alpha=0.4, cmap='RdBu_r', vmin=0, vmax=1)
legends = [plt.Rectangle((0,0),1,1,fc = pc.get_facecolor()[0]) for pc in contours.collections]
labels = [f'{a :.2f} - {b :.2f}' for a,b in zip(levels, levels[1:])]
sns.scatterplot(x='x0', y='x1', hue='y', ax=ax, data=data_1)
ax.legend(legends, labels, bbox_to_anchor=(1,1));

## [2] Second Dataset

The second dataset is superficially similar to the first one. The objective is still to predict whether the dependent variable ($y$) should be equal to 1 or 0 based on two independent variables ($x_0$ and $x_1$). However, it is designed to be a bit more challenging. You might want to explore it beyond what is provided in the cells below.

### [2.1] Load Data

This dataset can also be found in a .csv file in the same folder as this notebook.

In [ ]:
# Load second dataset and partition into train/test split
data_2 = pd.read_csv('data_2.csv')
data_2.head()

### [2.2] Split Data

The dataset is partitioned into two groups:

- `train` contains 500 samples that you should use to fit the model
- `test` contains another 500 samples that should only be used to check that your solution generalizes well to novel data

In [ ]:
data_2_train = data_2.query('split == "train"')
data_2_test = data_2.query('split == "test"')

# Partition data into independent (features) and depended (targets) variables
X_train, y_train = data_2_train[['x0', 'x1']], data_2_train['y']
X_test, y_test = data_2_test[['x0', 'x1']], data_2_test['y']

### [2.3] Fit and Evaluate Model

You may notice that the algorithm that worked pretty well on the first dataset comes up a bit short here. Feel free to add extra functionality to it and/or the data preprocessing pipeline that might improve performance. As a debugging reference; it is possible to obtain accuracies over 90%, both on the train and the test set.

In [ ]:
# Fit model (TO TRAIN SET ONLY)
model_2 = lr.LogisticRegression()  # <--- Feel free to add hyperparameters
model_2.fit(X_train, y_train)

# Calculate accuracy and cross entropy for insample predictions 
y_pred_train = model_2.predict(X_train)
print('Train')
print(f'Accuracy: {lr.binary_accuracy(y_true=y_train, y_pred=y_pred_train, threshold=0.5) :.3f}')
print(f'Cross Entropy:  {lr.binary_cross_entropy(y_true=y_train, y_pred=y_pred_train) :.3f}')

# Calculate accuracy and cross entropy for out-of-sample predictions
y_pred_test = model_2.predict(X_test)
print('\nTest')
print(f'Accuracy: {lr.binary_accuracy(y_true=y_test, y_pred=y_pred_test, threshold=0.5) :.3f}')
print(f'Cross Entropy:  {lr.binary_cross_entropy(y_true=y_test, y_pred=y_pred_test) :.3f}')

## [3] Further Steps (optional)

If you're done with the assignment but want to some more challenges; consider the following:

- Try a different objective function (e.g. [hinge-loss](https://en.wikipedia.org/wiki/Hinge_loss)). How does this affect the classifier?
- Try a more sophisticated stopping criterion than a fixed number of epochs
- Try a different optimization algorithm than simple gradient descent, for instance:
    - The [ADAM](https://optimization.cbe.cornell.edu/index.php?title=Adam) optimizer is a variation of gradient descent that is quite popular in deep learning.
    - The quasi-newton method [BGFS](https://optimization.cbe.cornell.edu/index.php?title=Quasi-Newton_methods#BFGS_method), which typically converge much faster
- Try to generalize your classifier to a [Multilayer Perceptron](https://en.wikipedia.org/wiki/Multilayer_perceptron) neural network classifier.

